In [1]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from fbprophet import Prophet
import pickle
import copy
from tqdm.auto import tqdm

In [2]:
import seaborn as sns
from plotly import express as px

In [3]:
from darts.models import *
from darts.models.forecasting import gradient_boosted_model
import darts.utils.timeseries_generation as tg
from darts.utils.likelihood_models import GaussianLikelihood
from darts.datasets import EnergyDataset
from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.timeseries import TimeSeries
import warnings

warnings.filterwarnings("ignore")

/ssd/jaewoo/anaconda3/envs/ts/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 765 ms, sys: 132 ms, total: 897 ms
Wall time: 897 ms


%%time
''' 스무딩 '''
win_size = 3


for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    
    #print()
    
    # 컬럼별 스무딩
    for col in fct_lst:
        series = df[col]
        rolling = series.rolling(window=win_size)
        rolling_median = rolling.median()
        df_dic[plc][col] = rolling_median
    df_dic[plc] = df_dic[plc].dropna()
    df_dic[plc] = df_dic[plc].iloc[-1000:,:]

df = df_dic[plc_lst[0]]
sns.lineplot(x = df.ds , y = df.pH)

df.pH.iloc[2800:]

In [5]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

---
# 예측 시작

In [16]:
dfall_comm= pickle.load( open('./mydata/dfall_comm.plk','rb'))

In [17]:
df_grp = dfall_comm.groupby('loc')

In [19]:
df_temp = df_grp.get_group(i)

In [21]:
df_temp[['dst', j]]

,dst,COD
364385,2015-04-23,NaN
364386,2015-04-24,NaN
364387,2015-04-25,NaN
364388,2015-04-26,NaN
364389,2015-04-27,NaN
...,...,...
365395,2018-01-27,1.560750
365396,2018-01-28,1.363361
365397,2018-01-29,0.968583
365398,2018-01-30,1.363361


In [12]:
from darts.models.forecasting import gradient_boosted_model, linear_regression_model

In [16]:
fct_df[y] = pd.to_numeric(fct_df.y)

0       0.115951
1       0.115951
2       0.115951
3       0.115951
4       0.115951
          ...   
2741   -0.136248
2742   -0.136248
2743   -0.136248
2744   -0.136248
2745   -0.136248
Name: y, Length: 2746, dtype: float64

In [41]:
# 추론 : lr
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10

        my_model = linear_regression_model.LinearRegressionModel(
        lags=lag,
        output_chunk_length=1,
        random_state= 2022)
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = float(forecast.yhat[idx])
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

  0%|          | 0/545 [00:00<?, ?it/s]

소요시간 : 15.182048459847769 minutes


In [42]:
# 추론 파일 저장
with open('./sub/baseline_LinearRegression_700.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)

In [44]:
# 추론 : lgbm
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = gradient_boosted_model.LightGBMModel(
        lags=lag,
        output_chunk_length=1,
        random_state= 2022)
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = forecast.yhat[idx]
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

  0%|          | 0/545 [00:00<?, ?it/s]

소요시간 : 29.06346660455068 minutes


In [45]:
# 추론 파일 저장
with open('./sub/baseline_lgbm700.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)

In [48]:
type(forecast.yhat[idx])

numpy.float64

In [49]:
import copy 

submit_ori = copy.deepcopy(submit)

In [50]:
for i in tqdm(plc_lst):  
    for j in fct_lst: # 피쳐 리스트
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(submit[i][k][j],6)

  0%|          | 0/545 [00:00<?, ?it/s]

In [51]:
# 추론 파일 저장
with open('./sub/baseline_lgbm700_r6.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)

---
## kalman

In [54]:
from darts.models.forecasting import kalman_forecaster

In [55]:
# 추론 : kalman
start_time = time.time()

for i in tqdm(plc_lst):
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        fct_df['ds'] = fct_df.ds.apply(lambda x : datetime.strptime(x , '%Y%m%d'))
        fct_df = fct_df.dropna()
        fct_df['y'] = pd.to_numeric(fct_df.y)
        
        df_align = pd.DataFrame([ x for x in list(pd.date_range( end = '2018-01-31', periods = len(fct_df), inclusive="both"))], columns=['ds'])
        df_align['y'] = fct_df.y.values
        
        # TimeSeris 로 변환 하기 
        dfts = TimeSeries.from_dataframe(df_align,  time_col = 'ds' , value_cols  = ['y'], fill_missing_dates=True  , freq='D')
        
        diff = datetime(2018,2,1) - df_align.loc[0].ds
        if diff.days > 720:
            lag = 700
        else:
            lag = diff.days - 10
            
        my_model = kalman_forecaster.KalmanForecaster()
        
        my_model.fit(dfts)
        pred_series = my_model.predict( n = 699 )
        
        forecast = pred_series.pd_dataframe()
        forecast['ds'] = forecast.index
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast.columns = ['yhat', 'ds']
        forecast = forecast.reset_index(drop=True)
        
        for idx,k in enumerate(submit[i].keys()):
            submit[i][k][j] = np.round(forecast.yhat[idx],6)
      
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

  0%|          | 0/545 [00:00<?, ?it/s]

소요시간 : 24.615875128904978 minutes


In [56]:
# 추론 파일 저장
with open('./sub/bs_kalman_r6.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)